# Import libraries

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests

# Set feature service variables

In [2]:
feat_srvc = 'https://services3.arcgis.com/0Fs3HcaFfvzXvm7w/ArcGIS/rest/services/CRIS_Zonal_Statistics_by_County/FeatureServer/5'
query = '/query'

# Use Geographic Identifiers (Field Name: GEOID) to pick which counties to calculate zonal statistics over.
uniqueID = 'GEOID'

# Pick counties, years, model sets, and models to return

In [3]:
# Pick the counties you want to process
# Put each county's GEOID between apostrophes and separate by commas, e.g. ids = '20201', '20117'
# Set ids to 'all' if using all disctrict IDs in the feature layer
ids = '20201', '20203', '20205'

In [4]:
# Pick the start and end year of the period you want to process
# If you only want to process 1 year, put your desired year for both the start and end
year_start = 2050
year_end = 2060

In [5]:
# Pick the model set you want to process
# Put each model set between apostrophes and separate by commas, e.g. 'STAR', 'LOCA2'
# Set model_sets to 'all' if you want to return all model sets
model_sets = 'LOCA2-STAR', 'STAR'

In [6]:
# Pick the specific models or the weighted ensemble of the model set you want to process
# Put each model between apostrophes and separate by commas, e.g. ids = '20201', '20117'
# NOTE: only for STAR and LOCA2. LOCA2STAR only includes the weighted ensemble
# Set models to 'all' if you want to return all models

# Choose from:
# 'Ensemble',
# 'ACCESS-CM2',
# 'ACCESS-ESM1-5',
# 'BCC-CSM2-MR',
# 'CanESM5',
# 'EC-Earth3',
# 'FGOALS-g3',
# 'GFDL-ESM4',
# 'INM-CM4-8',
# 'INM-CM5-0',
# 'IPSL-CM6A-LR',
# 'MIROC6',
# 'MPI-ESM1-2-HR',
# 'MPI-ESM1-2-LR',
# 'MRI-ESM2-0',
# 'NorESM2-LM',
# 'NorESM2-MM'

models = 'Ensemble', 'ACCESS-CM2'

In [7]:
if ids == 'all':
    
    id_condition = ""

elif np.size(ids) == 1:

    id_condition = f"{uniqueID} = '{ids}' AND"

elif np.size(ids) > 1:

    id_condition = f"{uniqueID} IN {ids} AND"


if model_sets == 'all':

    model_set_condition = ""

elif np.size(model_sets) == 1:

    model_set_condition = f"AND MODEL_SET = '{model_sets}'"
        
elif np.size(model_sets) > 1:
    
    model_set_condition = f"AND MODEL_SET IN {model_sets}"


if models == 'all':

    model_condition = ""

elif np.size(models) == 1:

    # model_condition = f"AND MODEL = '{models[0]}'"
    model_condition = f"AND MODEL = '{models}'"

elif np.size(models) > 1:

    model_condition = f"AND MODEL IN {models}"    

where_clause_id = f"{id_condition} YEAR >= {year_start} AND YEAR <= {year_end} {model_set_condition} {model_condition}"

# Pick variables and SSP scenarios to return

### Pick one of the three options below, run cells accordingly, then skip to "Retrieve data"

##### Note: Variables names include the SSP scenario as well.
##### Note: There is no SSP370 data for STAR.

##### Option 1: return all variables and all SSP scenarios

In [44]:
variables = '*'

##### Option 2: return manually selected subset of variables and SSP scenarios

In [ ]:
# Show available variables
response_temp = requests.get(feat_srvc + '?f=pjson')
data = response_temp.json()
fields = data['fields']
for field in fields:
    print('Variable:', field['name'], '\nDescription:', field['alias'], '\n')
    print()

In [8]:
# If you want 1 or more variables from the above list, select them here
# Put each variable between apostrophes and separate by commas, e.g. variables = 'TMAX_SSP245_MIN', 'TMIN_SSP585_MEAN'
variables = 'TMAX_SSP245_MIN', 'TMAX_SSP245_MEAN', 'TMAX_SSP245_MAX'

##### Option 3: return sets of variables.
##### For instance, all SSP245 variables or all 'Annual average daily maximum temperature' variables (TMAX_*) for all SSP scenarios

In [ ]:
# Run the line below for all SSP245 variables
# variables = [field['name'] for field in fields if "SSP245" in field['name']]

# Run the line below for all SSP245 and 'Annual average daily maximum temperature' variables (TMAX_*) variables
variables = [field['name'] for field in fields if "TMAX_" in field['name'] and "SSP245" in field['name']]

# Retrieve data

In [9]:
# add counties ('GEOID') and 'YEAR', MODEL_SET, and MODEL to the variables-to-return list
return_variables = ['GEOID', 'YEAR', 'MODEL_SET', 'MODEL']

if np.size(variables) == 1:

    return_variables.append(variables)

else:

    for var in variables:

        return_variables.append(var)

In [31]:
# Maximum number of variables that can be processed in each request: 50
# Maximum number of rows that can be processed in each request: 1000
# This code loops through variables and rows and appends each subset to a final GeoDataFrame (gdf)

# Number of variables that are requested to be processed
vars_to_process = len(return_variables)

# Number of variables to process in each loop
loop_size = 50

# Set initial value for loop number (v)
v = 0

# Loop through variables, 50 at one time
# While there are variables left to process, keep looping through this code
while vars_to_process > 0:

    # Sequentially subset the variables to process with 50 variables in each loop
    return_variables_condition = return_variables[max((loop_size*v - 1) + 1, 0): min(loop_size*(v+1), len(return_variables))]

    # Update number of variables to process, i.e. subtract 50 (loop_size)
    vars_to_process -= loop_size
    
    # Update loop number (v)
    v += 1

    # Set parameters for requests.get()
    params = {
        'where': where_clause_id,
        'outFields': return_variables_condition,
        'orderByFields': ['YEAR', 'GEOID'],
        'f': 'pgeojson',
    }

    # Request response from feature service
    response = requests.get(feat_srvc + query, params=params)


    # Put the first 50 variables in gdf to establish the GeoDataFrame
    if v == 1:
        
        # Translate the reponse.text into a temporary GeoDataFrame
        gdf_var1_temp = gpd.read_file(response.text)

        # Drop geometry field
        gdf_var1_temp.drop(columns=['geometry'], inplace=True)

        # Establish the final GeoDataFrame (gdf) and put the first subset of rows and variables (temporary GeoDataFrame) in it
        gdf = gdf_var1_temp


        # Set initial value for loop number (i)
        i = 0

        # Loop through rows, 1000 at one time
        while len(gdf_var1_temp) == 1000:
            
            # Update loop number (i)
            i = i + 1

            # Set the offset for in params. This says how many rows to skip from the start.
            offset = 1000 * i

            # Set parameters for requests.get()
            params = {
                'where': where_clause_id,
                'outFields': return_variables_condition,
                'orderByFields': ['YEAR', 'GEOID'],
                'f': 'pgeojson',
                'resultOffset': f'{offset}'
            }

            # Request response from feature service
            response = requests.get(feat_srvc + query, params=params)

            # Translate the reponse.text into a temporary GeoDataFrame
            gdf_var1_temp = gpd.read_file(response.text)

            # Concatenate the new subset to the final GeoDataFrame (gdf)
            gdf = pd.concat([gdf, gdf_var1_temp], ignore_index=True)

            # Drop geometry field
            gdf.drop(columns=['geometry'], inplace=True)


    # Put all the additional variables, beyond the first 50, in a temporary GeoDataFrame and concatenate with the final GeoDataFrame (gdf)
    elif v > 1:
    
        # Translate the reponse.text into a temporary GeoDataFrame
        gdf_var2_temp = gpd.read_file(response.text)
        
        # Drop geometry field
        gdf_var2_temp.drop(columns=['geometry'], inplace=True)

        # Establish the a temporart GeoDataFrame (gdf_var2) and put the first subset of rows and variables (temporary GeoDataFrame) in it     
        gdf_var2 = gdf_var2_temp


        # Set initial value for loop number (j)
        j = 0

        # Loop through rows, 1000 at one time
        while len(gdf_var2_temp) == 1000:

            # Update loop number (j)
            j = j + 1

            # Set the offset for in params. This says how many rows to skip from the start.
            offset = 1000 * j

            # Set parameters for requests.get()
            params = {
                'where': where_clause_id,
                'outFields': return_variables_condition,
                'orderByFields': ['YEAR', 'GEOID'],
                'f': 'pgeojson',
                'resultOffset': f'{offset}'
            }

            # Request response from feature service
            response = requests.get(feat_srvc + query, params=params)

            # Translate the reponse.text into a temporary GeoDataFrame
            gdf_var2_temp = gpd.read_file(response.text)

            # Concatenate the new subset to the temporary GeoDataFrame (gdf_var2)
            gdf_var2 = pd.concat([gdf_var2, gdf_var2_temp], ignore_index=True)

            # Drop geometry field
            gdf_var2.drop(columns=['geometry'], inplace=True)

        # Concatenate temporary GeoDataFrame (gdf_var2) to the final GeoDataFrame (gdf)
        gdf = pd.concat([gdf, gdf_var2], axis=1)

# Sort GeoDataFrame by GEOID, YEAR, MODEL_SET, and MODEL
gdf.sort_values(by=['GEOID', 'YEAR', 'MODEL_SET', 'MODEL'], inplace=True)

gdf

,GEOID,YEAR,MODEL_SET,MODEL,TMAX_SSP245_MIN,TMAX_SSP245_MEAN,TMAX_SSP245_MAX
0,20201,2050,LOCA2-STAR,Ensemble,69.51483,70.91604,71.77906
2,20201,2050,STAR,ACCESS-CM2,74.27699,75.38911,76.32729
1,20201,2050,STAR,Ensemble,69.59828,70.72666,71.67446
10,20201,2051,LOCA2-STAR,Ensemble,69.52514,70.92413,71.76859
11,20201,2051,STAR,ACCESS-CM2,74.61315,75.68138,76.69050
...,...,...,...,...,...,...,...
88,20205,2059,STAR,ACCESS-CM2,73.52187,74.05298,74.71010
87,20205,2059,STAR,Ensemble,73.01489,73.56829,74.26751
96,20205,2060,LOCA2-STAR,Ensemble,72.45110,73.12460,73.80055
98,20205,2060,STAR,ACCESS-CM2,77.32614,77.94305,78.45061


# Analysis examples

### Calculation 1: mean over time period

In [32]:
# If desired, run this cell to take a subset of the time period over which to calculate the mean
year_start_subset = 2054
year_end_subset = 2056

gdf1 = gdf[(gdf['YEAR'] >= year_start_subset) & (gdf['YEAR'] < year_end_subset+1)]

##### Run this cell to return mean of each variable in the GeoDataFrame

In [33]:
# Group by county (GEOID), MODEL_SET, and MODEL and calculate mean of period 'year_start_subset - year_end_subset'
variables_mean = gdf1.groupby(['GEOID', 'MODEL_SET', 'MODEL']).mean()

# Drop YEAR column
variables_mean.drop(columns=['YEAR'], inplace=True)

# If returning all variables, remove OBJECTID and BATCH_ID columns
if variables == '*':
    variables_mean.drop(columns=['OBJECTID', 'BATCH_ID'], inplace=True)

# Add index numbers to rows
variables_mean.reset_index(inplace=True)

# Rename column names to reflect that they represent the mean of each variable over time
variables_mean.rename(columns={gdf1.columns[i]: f"{gdf1.columns[i]} (mean)" for i in range(1, len(gdf1.columns))}, inplace=True)

# Show means of variables
variables_mean

,GEOID,MODEL_SET (mean),MODEL (mean),TMAX_SSP245_MIN (mean),TMAX_SSP245_MEAN (mean),TMAX_SSP245_MAX (mean)
0,20201,LOCA2-STAR,Ensemble,68.371870,69.761187,70.602940
1,20201,STAR,ACCESS-CM2,70.272800,71.311723,72.251793
2,20201,STAR,Ensemble,68.230797,69.355413,70.291220
3,20203,LOCA2-STAR,Ensemble,71.415987,71.705093,72.143013
4,20203,STAR,ACCESS-CM2,72.651997,73.341553,73.961023
5,20203,STAR,Ensemble,70.855900,71.501160,72.075060
6,20205,LOCA2-STAR,Ensemble,72.517187,73.206313,73.923377
7,20205,STAR,ACCESS-CM2,74.201110,74.831973,75.544087
8,20205,STAR,Ensemble,72.668843,73.224077,73.930683


### Calculation 2: change over time

In [34]:
# Set years over which to calculate the change
year_start_change = 2052
year_end_change = 2058

# Create two new GeoDataFrames which contain only the the start and end years, respectively
gdf_start_change = gdf[(gdf['YEAR'] == year_start_change)]
gdf_end_change = gdf[(gdf['YEAR'] == year_end_change)]

In [ ]:
# Set the indices of both GeoDataFrames to the counties (GEOIDs)
gdf_start_change.set_index('GEOID', inplace=True);
gdf_end_change.set_index('GEOID', inplace=True);

# Drop YEAR, MODEL_SET, and MODEL columns
gdf_start_change.drop(columns=['YEAR'], inplace=True)
gdf_end_change.drop(columns=['YEAR'], inplace=True)

# If returning all variables, remove OBJECTID and BATCH_ID columns
if variables == '*':
    gdf_start_change.drop(columns=['OBJECTID', 'BATCH_ID'], inplace=True)
    gdf_end_change.drop(columns=['OBJECTID', 'BATCH_ID'], inplace=True)

gdf_start_change.sort_values(by=['GEOID', 'MODEL_SET', 'MODEL'], inplace=True)
gdf_end_change.sort_values(by=['GEOID', 'MODEL_SET', 'MODEL'], inplace=True)

In [36]:
# Merge gdf_start_change and gdf_end_change and rename the variable columns by adding _start or _end, respectively
gdf_merged_change_absolute = gdf_start_change.merge(gdf_end_change, on=['GEOID', 'MODEL_SET', 'MODEL'], suffixes=('_start', '_end'))
gdf_merged_change_percentage = gdf_start_change.merge(gdf_end_change, on=['GEOID', 'MODEL_SET', 'MODEL'], suffixes=('_start', '_end'))

# Number of variables (or unique columns) for which to calculate the difference. Subtract 2 for MODEL_SET and MODEL. Divide by 2 because each variable has a start and end column
num_columns = int((np.shape(gdf_merged_change_absolute)[1] - 2) / 2)

# Loop through variables. Start at 2 to skip MODEL_SET and MODEL
for c in range(2, num_columns + 2):

    # Calculate difference (= end value - start value), for each variable, MODEL_SET, and MODEL. "c+num_columns" is to start at the end columns
    gdf_merged_change_absolute[f"{gdf_merged_change_absolute.columns[c].replace('_start', '')} (difference)"] = gdf_merged_change_absolute[f"{gdf_merged_change_absolute.columns[c+num_columns]}"] - gdf_merged_change_absolute[f"{gdf_merged_change_absolute.columns[c]}"]
    gdf_merged_change_percentage[f"{gdf_merged_change_percentage.columns[c].replace('_start', '')} (% difference)"] = ((gdf_merged_change_percentage[f"{gdf_merged_change_percentage.columns[c+num_columns]}"] - gdf_merged_change_percentage[f"{gdf_merged_change_percentage.columns[c]}"]) / gdf_merged_change_percentage[f"{gdf_merged_change_percentage.columns[c]}"]) * 100
    

# Drop the _start and _end columns. Only keep the difference columns
drop_columns = [col for col in gdf_merged_change_absolute.columns if '_start' in col or '_end' in col]
variables_difference_absolute = gdf_merged_change_absolute.drop(columns=drop_columns)
variables_difference_percentage = gdf_merged_change_percentage.drop(columns=drop_columns)

In [37]:
# Show absolute difference of each variable between the start and end years
variables_difference_absolute

,MODEL_SET,MODEL,TMAX_SSP245_MIN (difference),TMAX_SSP245_MEAN (difference),TMAX_SSP245_MAX (difference)
GEOID,,,,,
20201,LOCA2-STAR,Ensemble,1.57041,1.55212,1.54677
20201,STAR,ACCESS-CM2,-3.36194,-3.30668,-3.31016
20201,STAR,Ensemble,1.58960,1.60839,1.62089
20203,LOCA2-STAR,Ensemble,1.33129,1.34856,1.37189
20203,STAR,ACCESS-CM2,-2.89496,-2.85110,-2.89675
20203,STAR,Ensemble,1.51294,1.51642,1.52237
20205,LOCA2-STAR,Ensemble,1.25465,1.21443,1.17534
20205,STAR,ACCESS-CM2,-2.50302,-2.53347,-2.39778
20205,STAR,Ensemble,1.28699,1.24714,1.21302


In [38]:
# Show percentage difference of each variable between the start and end years
variables_difference_percentage

,MODEL_SET,MODEL,TMAX_SSP245_MIN (% difference),TMAX_SSP245_MEAN (% difference),TMAX_SSP245_MAX (% difference)
GEOID,,,,,
20201,LOCA2-STAR,Ensemble,2.317798,2.245104,2.210008
20201,STAR,ACCESS-CM2,-4.671881,-4.526916,-4.475508
20201,STAR,Ensemble,2.350263,2.340246,2.327123
20203,LOCA2-STAR,Ensemble,1.878801,1.895667,1.917223
20203,STAR,ACCESS-CM2,-3.902423,-3.813903,-3.846038
20203,STAR,Ensemble,2.152105,2.137874,2.129351
20205,LOCA2-STAR,Ensemble,1.747475,1.675693,1.606024
20205,STAR,ACCESS-CM2,-3.329445,-3.343293,-3.139514
20205,STAR,Ensemble,1.788614,1.720210,1.657266
